# Preprocesamiento de datos

## OBJETIVO

### Predicción de la volatilidad histórica

* Inputs: Lags del histórico de retornos logarítmicos del precio de Bitcoin.
* Output: la volatilidad futura de 7 días


In [1]:
import pandas as pd
import numpy as np

# Ventanas de tiempo 
ventanas = [7,14,21,28]

# Lectura de archivo y selección de variables de interés
btc = pd.read_csv(r"C:\Users\Hp\MACHINE\MINI_PRY_2\data\btc_1d_data_2018_to_2025.csv")
btc['Date'] = btc['Open time']
btc = btc[['Date', 'Close']].dropna().reset_index(drop=True)
btc['Date'] = pd.to_datetime(btc['Date'], errors='coerce', format='%Y-%m-%d')
btc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2594 entries, 0 to 2593
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   Date    2594 non-null   datetime64[ns]
 1   Close   2594 non-null   float64       
dtypes: datetime64[ns](1), float64(1)
memory usage: 40.7 KB


## 1. Cálculo de retornos logarítmicos

Un retorno logarítmico $r_t$ se define como:
$$
r_t = ln(\frac{P_t}{P_{t-1}})
$$
donde $P_t$ es el precio de cierre en el día $t$.

In [2]:
# 1. Calcular retornos logarítmicos
btc["log_return"] = np.log(btc["Close"] / btc["Close"].shift(1))

## 2. Volatilidad histórica (rolling std)

* Cálculo de la volatilidad histórica

La volatilidad es una medida de dispersión de los rendimientos de un activo financiero. 

Para una ventana de tamaño $n$:
$$
\sigma_t = \sqrt{\frac{1}{n} \sum_{i=1}^{n} (r_{t-i}-\hat{r})^2}
$$

donde $r_t$ es el retorno logarítmico, $\hat{r}$ el promedio de retornos en una ventana y $\sigma_t$ es el promedio de retornos en la ventana. 

En este análisis se tomaran en cuenta 4 ventanas de interés: 7, 14, 21 y 28 días. 

Variables: `volatility_7d, volatility_14d, volatility_21d, volatility_28d`

Qué es: desviación estándar de los retornos en una ventana móvil.

Por qué lo usamos:

Captura riesgo y dinámica de la variabilidad.

In [3]:
# 2. Calcular volatilidades históricas (features)
for v in ventanas:
    btc[f"Volatil_D{v}"] = btc["log_return"].rolling(window=v).std() * np.sqrt(365)


Para facilitar la comparación entre activos, para cada ventana se anualizaron los valores multiplicándolos por la raíz cuadrada de 365 (días en un año).

##  6. Data procesada

Luego de crear las variables requeridas para el análisis, se eliminan las filas con datos faltantes, como consecuencia de la imposibilidad de calcular recuentos históricos al ser los primeros registros.

In [4]:
btc.columns

Index(['Date', 'Close', 'log_return', 'Volatil_D7', 'Volatil_D14',
       'Volatil_D21', 'Volatil_D28'],
      dtype='object')

```{note}
Los valores faltantes resultan de la imposibilidad de calcular registros históricos para los primeros registros

In [5]:
btc.isna().sum().sort_values(ascending=False).head()

Volatil_D28    28
Volatil_D21    21
Volatil_D14    14
Volatil_D7      7
log_return      1
dtype: int64

In [6]:
# Eliminar filas con NA
na_filas = btc.isna().any(axis=1).sum()
print(f"Filas eliminadas por contener al menos un NA: {na_filas}")
print(f"Representa un {round((na_filas/len(btc))*100, 3)} % del total de datos")
btc_clean = btc.dropna()

Filas eliminadas por contener al menos un NA: 28
Representa un 1.079 % del total de datos


```{note}
De esta manera se obtiene un dataset de 2558 registros resumidos en 21 variables.

In [7]:
btc_clean.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2566 entries, 28 to 2593
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   Date         2566 non-null   datetime64[ns]
 1   Close        2566 non-null   float64       
 2   log_return   2566 non-null   float64       
 3   Volatil_D7   2566 non-null   float64       
 4   Volatil_D14  2566 non-null   float64       
 5   Volatil_D21  2566 non-null   float64       
 6   Volatil_D28  2566 non-null   float64       
dtypes: datetime64[ns](1), float64(6)
memory usage: 160.4 KB


In [8]:
btc_clean.to_csv("BTC_all.csv", index=False)